In [22]:
from collections import defaultdict, Counter
import json
from itertools import product

Create Node data from event_types and processed edges.

Create Edge data from event_log.

In [23]:
journey_rawdata = open("data/journey.csv").readlines()
print(journey_rawdata[:5])

['1624,9\n', '1624,5\n', '1624,9\n', '1624,40\n', '1624,40\n']


In [24]:
groups = json.load(open("data/event_groupings.json"))

In [25]:
journeys = defaultdict(list)
for line in journey_rawdata:
    customer, event = line.split(',')
    event = groups["Type_to_group"][event[:-1]]
    customer = int(customer)
    journeys[customer].append(event)

In [26]:
journey_keys = iter(journeys.keys())
journeys[next(journey_keys)]

[0, 4]

In [27]:
journeys[next(journey_keys)]

[0, 4, 2, 4, 2]

In [28]:
def pairs(events, length_cap):
    events = [(0, x[1]) if x[1] == 2 else x for x in enumerate(events[:length_cap])]  # only 1 cancellation event
    if (0, 2) in events:
        events = events[:events.index((0,2)) + 1]
    return [(events[i], events[i+1]) for i in range(len(events) - 1)]

# >> pairs([1,5,5,27,3],2)
# [((0, 1), (1, 5))]

1. get set of pairs that actually occur - looks like all or close. XX
1. output to a file and record indices, ie create dict{pair: index}
1. create counter of union of journeys: dict(pair of pairs : count) XX
1. use lookup dict to map keys to dict(pair of indices : count)
1. output

What about subscription status? What can we glean from this file?

Definitely need to create groupings of event_types.

```
enumerate the possible nodes (0-20, 0-13), 260 of them. Gives:
(index, (event_number, event_group))
create func to create json for (index, (event_clock, event_group)) -> [{name: groups["Groups"][event_group]}]
create dict:
{(event_number, event_group): index}
use dict to map edges:
(((num, grp), (num, grp)), count) -> {source: index, target: index, value: count}
```

In [29]:
def endpoints(segments):
    endpoints = set()
    for seg in segments:
        endpoints.add(seg[0])
        endpoints.add(seg[1])
    return endpoints

In [30]:
num_events = 10

journey_segments = []  # [((a, b), (c, d))]
nodeset = set()
for j in journeys.values():
    if j[0] == 0:
        segments = pairs(j, num_events)
        nodeset = nodeset.union(endpoints(segments))
        journey_segments.extend(segments)

In [31]:
c = Counter(journey_segments)

In [35]:
possible_node_ids = list(enumerate([x for x in product(range(num_events), range(14)) if x in nodeset]))
nodes = [{"name": groups["Groups"][node[1][1]]} for node in possible_node_ids] #  + "-" + str(node[1][0])
node_lookup = {node[1]:node[0] for node in possible_node_ids}

In [36]:
edges = [{"source": node_lookup[k[0]], "target": node_lookup[k[1]], "value": v} for k,v in c.items()]

In [37]:
graph = {"nodes": nodes, "links": edges}
json.dump(graph, open("data/graph.json", "w"), indent=2)